In [1]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 98% 340M/346M [00:07<00:00, 18.4MB/s]
100% 346M/346M [00:07<00:00, 49.9MB/s]


In [2]:
import zipfile
zip_ref = zipfile.ZipFile('./intel-image-classification.zip', 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

In [3]:
import os

base_dir = 'tmp'

train_dir_0 = 'seg_train/seg_train'
valid_dir_0 = 'seg_test/seg_test'

train_dir = os.path.join(base_dir, train_dir_0)
valid_dir = os.path.join(base_dir, valid_dir_0)

In [4]:
building_train_dir = os.path.join(train_dir, 'buildings')
forest_train_dir = os.path.join(train_dir, 'forest')
glacier_train_dir = os.path.join(train_dir, 'glacier')
mountain_train_dir = os.path.join(train_dir, 'mountain')
sea_train_dir = os.path.join(train_dir, 'sea')
street_train_dir = os.path.join(train_dir, 'street')

building_valid_dir = os.path.join(valid_dir, 'buildings')
forest_valid_dir = os.path.join(valid_dir, 'forest')
glacier_valid_dir = os.path.join(valid_dir, 'glacier')
mountain_valid_dir = os.path.join(valid_dir, 'mountain')
sea_valid_dir = os.path.join(valid_dir, 'sea')
street_valid_dir = os.path.join(valid_dir, 'street')

In [5]:
print(len(os.listdir(building_train_dir)) + len(os.listdir(forest_train_dir))  + len(os.listdir(glacier_train_dir)) + len(os.listdir(mountain_train_dir)) + len(os.listdir(sea_train_dir)) + len(os.listdir(street_train_dir)))

14034


In [6]:
print(len(os.listdir(building_valid_dir)) + len(os.listdir(forest_valid_dir)) + len(os.listdir(glacier_valid_dir)) + len(os.listdir(mountain_valid_dir)) + len(os.listdir(sea_valid_dir)) + len(os.listdir(street_valid_dir)))

3000


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical'
)

valid_datagen = ImageDataGenerator(rescale = 1/255)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'categorical'
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [9]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [10]:
from tensorflow.keras.optimizers import RMSprop
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = RMSprop(learning_rate = 0.001),
    metrics = ['accuracy']
)

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch = 110,
    epochs = 25,
    verbose = 1,
    validation_data = valid_generator,
    validation_steps = 24
)

Epoch 1/25
110/110 [==============================] - 101s 829ms/step - loss: 1.5306 - accuracy: 0.3741 - val_loss: 1.1100 - val_accuracy: 0.5543
Epoch 2/25
110/110 [==============================] - 89s 813ms/step - loss: 1.2070 - accuracy: 0.5217 - val_loss: 1.0839 - val_accuracy: 0.5610
Epoch 3/25
110/110 [==============================] - 88s 803ms/step - loss: 1.0879 - accuracy: 0.5807 - val_loss: 0.8819 - val_accuracy: 0.6817
Epoch 4/25
110/110 [==============================] - 89s 810ms/step - loss: 0.9952 - accuracy: 0.6218 - val_loss: 0.8340 - val_accuracy: 0.6867
Epoch 5/25
110/110 [==============================] - 89s 807ms/step - loss: 0.9324 - accuracy: 0.6464 - val_loss: 0.7942 - val_accuracy: 0.7117
Epoch 6/25
110/110 [==============================] - 90s 813ms/step - loss: 0.8805 - accuracy: 0.6689 - val_loss: 0.6795 - val_accuracy: 0.7487
Epoch 7/25
110/110 [==============================] - 90s 814ms/step - loss: 0.8373 - accuracy: 0.6906 - val_loss: 0.7176 - val_a

In [37]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# URL of the image you want to predict (replace with your image URL)
image_url = 'https://plus.unsplash.com/premium_photo-1681550090719-5754576792a8?w=600&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MTN8fGJ1aWxkaW5nfGVufDB8fDB8fHww'

# Download the image using requests
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Define preprocessing parameters (must match your training preprocessing)
img_width, img_height = 150, 150  # Example dimensions, adjust as per your model

# Resize and preprocess the image
img = img.resize((img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create batch of size 1

# Create an instance of ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1./255)  # Ensure the same normalization as during training

# Preprocess the image using the datagen
img_preprocessed = datagen.standardize(img_array)


In [38]:
# Predict probabilities for each class
predictions = model.predict(img_preprocessed)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map predicted index to class label (replace with your actual class labels)
class_labels = ['building', 'forest', 'glacier', 'mountain', 'sea', 'street']
predicted_class = class_labels[predicted_class_index]

# Print the predicted class and probabilities
print(f'Predicted Class: {predicted_class}')
print(f'Probabilities: {predictions[0]}')

# If you want to get the top-k predicted classes and their probabilities:
top_k = 3  # Number of top classes to display
top_k_indices = np.argsort(predictions[0])[::-1][:top_k]
for i, idx in enumerate(top_k_indices):
    print(f'Top-{i+1} Prediction: Class {class_labels[idx]} with probability {predictions[0][idx]}')


1/1 [==============================] - 0s 17ms/step
Predicted Class: building
Probabilities: [0.9476509  0.0703125  0.28590718 0.49648842 0.01397119 0.08000156]
Top-1 Prediction: Class building with probability 0.9476509094238281
Top-2 Prediction: Class mountain with probability 0.49648842215538025
Top-3 Prediction: Class glacier with probability 0.28590717911720276
